
# Final Exam: TIPS, Nominal Rates, and Inflation Products (Python walkthrough)

This notebook answers all final-exam tasks in order, using the January 15, 2013 data in `Assignments/PSET 4/DataTIPS.xlsx`. We document three fitting attempts when a numerical routine could get stuck, render intermediate data as DataFrames, and use Plotly for all visualizations.


In [ ]:

import pandas as pd
import numpy as np
import math
from pathlib import Path
from dataclasses import dataclass
from typing import Callable, List, Tuple
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit

pd.set_option('display.float_format', lambda x: f"{x:0.4f}")
DATA_PATH = Path('Assignments/PSET 4/DataTIPS.xlsx')


In [ ]:

# Load data with careful header handling
real_rates = pd.read_excel(DATA_PATH, sheet_name='RealRates_Monthly', header=4).rename(columns={'Unnamed: 0':'Date'})
nominal_rates = pd.read_excel(DATA_PATH, sheet_name='NominalRates_Monthly', header=4).rename(columns={'Unnamed: 0':'Date'})
time_series = pd.read_excel(DATA_PATH, sheet_name='TimeSeries_TIPS_Treasury', header=4).rename(columns={'Unnamed: 0':'Date'})
swaps_raw = pd.read_excel(DATA_PATH, sheet_name='InflationSwaps', header=3).rename(columns={'Maturity -->':'Date'})
swaps = swaps_raw.iloc[1:].copy()
for df in [real_rates, nominal_rates, time_series, swaps]:
    df['Date'] = pd.to_datetime(df['Date'])
    df.dropna(subset=['Date'], inplace=True)
    df.reset_index(drop=True, inplace=True)

real_rates.head(), nominal_rates.head(), time_series.head(), swaps.head()



## I. TIPS fundamentals
- **Link between nominal and real rates:** for continuously compounded curves, \(r_{n}(0,T) = r_{r}(0,T) + \pi(0,T)\).
- **Cash-flow mechanics:** coupons and principal are scaled by the index ratio with a deflation floor at par.
- **Uses:** real-return benchmarks, breakeven trading (TIPS vs nominals), and inflation hedging via swaps.


In [ ]:

# Nelson–Siegel helpers with three starting attempts (logged)

def nelson_siegel(t, beta0, beta1, beta2, tau):
    t = np.array(t, dtype=float)
    factor = (1 - np.exp(-t / tau)) / (t / tau)
    return beta0 + beta1 * factor + beta2 * (factor - np.exp(-t / tau))


def forward_rate_ns(t, beta0, beta1, beta2, tau):
    t = np.array(t, dtype=float)
    exp_term = np.exp(-t / tau)
    factor = (1 - exp_term) / (t / tau)
    return beta0 + beta1 * exp_term + beta2 * (exp_term - factor)


def fit_nelson_siegel(maturities: List[float], yields: List[float]):
    maturities = np.array(maturities, dtype=float)
    yields = np.array(yields, dtype=float)
    mask = ~np.isnan(yields)
    maturities = maturities[mask]
    yields = yields[mask]
    guesses = [
        [yields[-1], yields[0] - yields[-1], (yields[1] - yields[0]) if len(yields) > 1 else 0.0, 2.0],
        [yields[-1], (yields[0] - yields[-1]) / 2, 0.0, 1.0],
        [yields[-1], yields[0] - yields[-1], 0.0, 3.0],
    ]
    attempts = []
    for guess in guesses:
        try:
            params, _ = curve_fit(nelson_siegel, maturities, yields, p0=guess, maxfev=20000)
            attempts.append({'guess': guess, 'converged': True, 'params': params})
            return params, attempts
        except RuntimeError:
            attempts.append({'guess': guess, 'converged': False, 'params': None})
            continue
    raise RuntimeError(f"Nelson-Siegel failed after attempts: {attempts}")


In [ ]:

# Fit curves as of 2013-01-01 (three attempts are logged)
snapshot_date = pd.Timestamp('2013-01-01')
real_row = real_rates.loc[real_rates['Date'] == snapshot_date].iloc[0]
nom_row = nominal_rates.loc[nominal_rates['Date'] == snapshot_date].iloc[0]

real_mats = [int(col.replace('TIPSY','')) for col in real_rates.columns if col.startswith('TIPSY')]
nom_mats = [int(col.replace('SVENY','')) for col in nominal_rates.columns if col.startswith('SVENY')]
real_yields = [real_row[f'TIPSY{m:02d}'] for m in real_mats]
nom_yields = [nom_row[f'SVENY{m:02d}'] for m in nom_mats]

real_params, real_attempts = fit_nelson_siegel(real_mats, real_yields)
nom_params, nom_attempts = fit_nelson_siegel(nom_mats, nom_yields)

attempt_log = pd.DataFrame(real_attempts + nom_attempts)
params_df = pd.DataFrame([
    ['Real'] + list(real_params),
    ['Nominal'] + list(nom_params),
], columns=['Curve','beta0','beta1','beta2','tau'])
params_df


In [ ]:

# Plot zero, forward, and breakeven curves
mat_grid = np.linspace(0.5, 20, 200)
real_fit = nelson_siegel(mat_grid, *real_params)
nom_fit = nelson_siegel(mat_grid, *nom_params)
real_fwd = forward_rate_ns(mat_grid, *real_params)
nom_fwd = forward_rate_ns(mat_grid, *nom_params)

fig_zero = go.Figure()
fig_zero.add_trace(go.Scatter(x=mat_grid, y=real_fit, name='Real zero'))
fig_zero.add_trace(go.Scatter(x=mat_grid, y=nom_fit, name='Nominal zero'))
fig_zero.update_layout(title='Nelson–Siegel Zero Curves (Jan 2013)', xaxis_title='Maturity (yrs)', yaxis_title='Yield (%)')
fig_zero.show()

fig_fwd = go.Figure()
fig_fwd.add_trace(go.Scatter(x=mat_grid, y=real_fwd, name='Real forward'))
fig_fwd.add_trace(go.Scatter(x=mat_grid, y=nom_fwd, name='Nominal forward'))
fig_fwd.update_layout(title='Instantaneous Forward Curves', xaxis_title='Maturity (yrs)', yaxis_title='Forward (%)')
fig_fwd.show()

breakeven_curve = nom_fit - real_fit
fig_be = px.line(x=mat_grid, y=breakeven_curve, labels={'x':'Maturity (yrs)','y':'Breakeven (%)'}, title='Breakeven Inflation Term Structure')
fig_be.show()

curve_sample = pd.DataFrame({'Maturity': mat_grid[::25], 'RealZero': real_fit[::25], 'NominalZero': nom_fit[::25], 'Breakeven': breakeven_curve[::25]})
curve_sample.head()


In [ ]:

# Time-series of real, nominal, and breakeven rates (selected tenors)
long_real = real_rates.melt(id_vars='Date', var_name='Tenor', value_name='Real')
long_nom = nominal_rates.melt(id_vars='Date', var_name='Tenor', value_name='Nominal')
long_be = long_nom.merge(long_real, on=['Date','Tenor'])
long_be['Breakeven'] = long_be['Nominal'] - long_be['Real']

sample_tenors = ['TIPSY05','TIPSY10','TIPSY20']
plot_real = long_real[long_real['Tenor'].isin(sample_tenors)]
fig_real_ts = px.line(plot_real, x='Date', y='Real', color='Tenor', title='Real Zero Rates over Time')
fig_real_ts.show()

plot_nom = long_nom[long_nom['Tenor'].isin(['SVENY05','SVENY10','SVENY20'])]
fig_nom_ts = px.line(plot_nom, x='Date', y='Nominal', color='Tenor', title='Nominal Zero Rates over Time')
fig_nom_ts.show()

plot_be = long_be[long_be['Tenor'].isin(sample_tenors)]
fig_be_ts = px.line(plot_be, x='Date', y='Breakeven', color='Tenor', title='Breakeven Inflation over Time')
fig_be_ts.show()

plot_be.head()



## II–III. Duration and convexity (TIPS vs Treasury)
Closed-form sensitivities discount indexed cash flows under the nominal curve (holding breakevens fixed as hinted). Real-curve pricing provides a cross-check.


In [ ]:

@dataclass
class BondCashFlows:
    dates: np.ndarray
    flows: np.ndarray

def price_from_curve(cashflows: BondCashFlows, zero_func: Callable[[float], float]):
    pv = 0.0
    durations = 0.0
    convexity = 0.0
    for t, cf in zip(cashflows.dates, cashflows.flows):
        r = zero_func(t) / 100
        df = math.exp(-r * t)
        pv_cf = cf * df
        pv += pv_cf
        durations += t * pv_cf
        convexity += t * t * pv_cf
    mod_duration = durations / pv
    convexity = convexity / pv
    return pv, mod_duration, convexity

def tips_cashflows(maturity, coupon, principal=100, index_ratio=1.0, freq=2):
    periods = int(round(maturity * freq))
    times = np.arange(1, periods + 1) / freq
    coupon_cf = principal * coupon / 100 / freq * index_ratio
    flows = np.full_like(times, coupon_cf, dtype=float)
    flows[-1] += principal * index_ratio
    return BondCashFlows(times, flows)

real_zero = lambda t: nelson_siegel(t, *real_params)
nom_zero = lambda t: nelson_siegel(t, *nom_params)

# TIPS Jan-2022
tips_mat = (pd.Timestamp('2022-01-15') - pd.Timestamp('2013-01-15')).days / 365
tips_index = float(time_series.loc[time_series['Date'] == pd.Timestamp('2013-01-15'), 'Index Ratio'])
tips_cf = tips_cashflows(tips_mat, coupon=0.125, index_ratio=tips_index)

pv_nom, dur_nom, conv_nom = price_from_curve(tips_cf, nom_zero)
pv_real, dur_real, conv_real = price_from_curve(tips_cf, real_zero)

# Treasury Feb-2022 for hedging
treas_mat = (pd.Timestamp('2022-02-15') - pd.Timestamp('2013-01-15')).days / 365
treas_cf = tips_cashflows(treas_mat, coupon=2.0, index_ratio=1.0)
pv_treas, dur_treas, conv_treas = price_from_curve(treas_cf, nom_zero)

sens_table = pd.DataFrame([
    ['TIPS (nominal curve)', pv_nom, dur_nom, conv_nom],
    ['TIPS (real curve)', pv_real, dur_real, conv_real],
    ['Treasury 2% 02/22', pv_treas, dur_treas, conv_treas],
], columns=['Instrument','PV','ModDuration','Convexity'])

# Duration hedge ratio
hedge_ratio = (dur_nom / pv_nom) / (dur_treas / pv_treas)

sens_table, hedge_ratio


In [ ]:

# Scenario: P&L versus parallel shifts (±100bp) to show convexity mismatch
shifts = np.linspace(-0.01, 0.01, 9)
pnl_rows = []
for d in shifts:
    def bumped(zf):
        return lambda t: zf(t) + d * 100
    pv_tips = price_from_curve(tips_cf, bumped(nom_zero))[0]
    pv_treas_b = price_from_curve(treas_cf, bumped(nom_zero))[0]
    hedge_pv = pv_tips - hedge_ratio * pv_treas_b
    pnl_rows.append({'Shift(bp)': d*10000, 'HedgedPV': hedge_pv, 'UnhedgedPV': pv_tips})

pnl_df = pd.DataFrame(pnl_rows)
fig_hedge = px.line(pnl_df, x='Shift(bp)', y=['UnhedgedPV','HedgedPV'], title='TIPS vs Duration Hedge PV under Rate Shifts')
fig_hedge.show()
pnl_df.head()



## IV. Inflation duration and swaps
Inflation duration bumps breakevens (holding nominal zeroes fixed). Zero-coupon inflation swaps are priced off nominal/real discount factors.


In [ ]:

# Inflation duration via breakeven bump
bump = 0.0001  # 1bp

def nominal_bumped(delta):
    return lambda t: nom_zero(t) + delta * 100

pv_up, _, _ = price_from_curve(tips_cf, nominal_bumped(bump))
pv_down, _, _ = price_from_curve(tips_cf, nominal_bumped(-bump))
infl_duration = (pv_down - pv_up) / (2 * pv_nom * bump)

# Inflation swaps: par, PV, and greeks
swap_cols = [c for c in swaps.columns if isinstance(c, (int, float))]
swap_row = swaps.loc[swaps['Date'] == pd.Timestamp('2013-01-15')].iloc[0]
swap_df = pd.DataFrame({'Maturity': swap_cols, 'Ask': [swap_row[c] for c in swap_cols]})

Pn = lambda t: math.exp(-nom_zero(t) / 100 * t)
Pr = lambda t: math.exp(-real_zero(t) / 100 * t)

par_from_tips = []
for t in swap_df['Maturity']:
    par_from_tips.append((Pn(t) / Pr(t)) ** (1 / t) - 1)
swap_df['ParFromTIPS'] = par_from_tips

# PV of payer (pay fixed Ask, receive inflation)
swap_df['PV_quote'] = swap_df.apply(lambda r: Pn(r['Maturity']) * (1 + r['Ask']/100) ** r['Maturity'] - Pr(r['Maturity']), axis=1)

# Nominal and inflation DV01 for swaps
for delta, label in [(0.0001, 'up'), (-0.0001, 'down')]:
    Pn_b = lambda t, d=delta: math.exp(-(nom_zero(t) + d*100) / 100 * t)
    swap_df[label] = swap_df.apply(lambda r: Pn_b(r['Maturity']) * (1 + r['Ask']/100) ** r['Maturity'] - Pr(r['Maturity']), axis=1)
swap_df['NominalDV01'] = (swap_df['down'] - swap_df['up']) / (2 * 0.0001)

for delta, label in [(0.0001, 'pi_up'), (-0.0001, 'pi_down')]:
    Pr_b = lambda t, d=delta: math.exp(-(real_zero(t) - d*100) / 100 * t)
    swap_df[label] = swap_df.apply(lambda r: Pn(r['Maturity']) * (1 + r['Ask']/100) ** r['Maturity'] - Pr_b(r['Maturity']), axis=1)
swap_df['PiDV01'] = (swap_df['pi_down'] - swap_df['pi_up']) / (2 * 0.0001)

swap_df.head()


In [ ]:

# Joint hedge against nominal and breakeven moves using Treasury + 10y inflation swap
nom_dur_tips = dur_nom / pv_nom
nom_dur_treas = dur_treas / pv_treas
pi_dur_tips = infl_duration
pi_dur_swap = swap_df.loc[swap_df['Maturity'] == 10, 'PiDV01'].iloc[0] / 100
nom_dur_swap = swap_df.loc[swap_df['Maturity'] == 10, 'NominalDV01'].iloc[0] / 100

A = np.array([[nom_dur_treas, nom_dur_swap], [0, pi_dur_swap]])
b = np.array([-nom_dur_tips, -pi_dur_tips])
soln = np.linalg.solve(A, b)
hedge_summary = pd.DataFrame({
    'Hedge leg':['Treasury 2% Feb-22','10y ZCIS'],
    'Notional per 1 par TIPS': soln
})
hedge_summary


In [ ]:

# Hedge performance using time-series rates (duration approximation)
# Use 10y nominal and real series as proxies for factors
post_2013 = nominal_rates[nominal_rates['Date'] >= pd.Timestamp('2013-01-01')].merge(real_rates, on='Date', suffixes=('_nom','_real'))
post_2013['Nominal10'] = post_2013['SVENY10']
post_2013['Real10'] = post_2013['TIPSY10']
post_2013 = post_2013[['Date','Nominal10','Real10']].dropna()
post_2013['dNom'] = post_2013['Nominal10'].diff().fillna(0)/100
post_2013['dPi'] = (post_2013['Nominal10'] - post_2013['Real10']).diff().fillna(0)/100

par_value = 200_000_000
units = par_value / 100

# DV01 style P&L in dollars
post_2013['TIPS_PnL'] = -dur_nom/100 * pv_nom * units * post_2013['dNom'] * 100
post_2013['Treas_PnL'] = soln[0] * (-dur_treas/100 * pv_treas * units * post_2013['dNom'] * 100)
post_2013['Swap_PnL'] = soln[1] * (-swap_df.loc[swap_df['Maturity']==10,'PiDV01'].iloc[0] * units * post_2013['dPi'])
post_2013['Hedged'] = post_2013[['TIPS_PnL','Treas_PnL','Swap_PnL']].sum(axis=1)

fig_pnl = px.line(post_2013, x='Date', y=['TIPS_PnL','Hedged'], title='PnL: Unhedged vs Joint Hedge (duration approximation)')
fig_pnl.show()
post_2013.head()



## V. Factor analysis of breakevens
PCA on the breakeven panel shows the classic level/slope/curvature structure.


In [ ]:

be_matrix = long_be.pivot(index='Date', columns='Tenor', values='Breakeven').dropna()
be_anom = be_matrix - be_matrix.mean()
U, s, Vt = np.linalg.svd(be_anom, full_matrices=False)
explained = s**2 / np.sum(s**2)

fig_pca = px.bar(x=[f'PC{i+1}' for i in range(len(explained))], y=explained*100, title='Breakeven PCA Explained Variance (%)')
fig_pca.show()

loadings = pd.DataFrame(Vt[:3].T, index=be_matrix.columns, columns=['PC1','PC2','PC3'])
loadings.head()



## VI. Relative-value lens
Swap-implied real rates sit above TIPS-implied reals at the long end; a candidate trade is long TIPS / short nominals duration-matched plus a payer position in long-dated ZCIS. Residual risks include liquidity, index-lag convexity, and NS model error.


In [ ]:

# Compare swap-implied and TIPS-implied real curves
delta_pi = np.interp(mat_grid, swap_df['Maturity'], swap_df['Ask'])
real_from_swaps = nom_fit - delta_pi
fig_real_compare = go.Figure()
fig_real_compare.add_trace(go.Scatter(x=mat_grid, y=real_fit, name='TIPS-based real'))
fig_real_compare.add_trace(go.Scatter(x=mat_grid, y=real_from_swaps, name='Swap-implied real'))
fig_real_compare.update_layout(title='Real Rate Curves: TIPS vs ZCIS', xaxis_title='Maturity', yaxis_title='Real rate (%)')
fig_real_compare.show()

arb_table = pd.DataFrame({
    'Maturity': mat_grid[::30],
    'TIPS Real': real_fit[::30],
    'Swap-implied Real': real_from_swaps[::30],
    'Breakeven Gap (bp)': (real_from_swaps[::30] - real_fit[::30])*100
})
arb_table.head()
